In [ ]:
import pandas as pd
import os
from cac.utils.io import read_yml
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
import pickle
import argparse
from sklearn.preprocessing import LabelEncoder

### Play Around with Preprocessing

In [30]:
attributes = pd.read_csv(f'/data/wiai-facility/processed/attributes.csv')

median_days = dict()
symptoms = ['cough', 'fever', 'shortness_of_breath']
for symptom in symptoms:
    median_days[symptom] = attributes[attributes[f'enroll_{symptom}'] == 'Yes'][f'enroll_days_with_{symptom}'].median()
    attributes.loc[attributes[f'enroll_{symptom}'] == 'No', f'enroll_days_with_{symptom}'] = 0
    attributes[f'enroll_days_with_{symptom}'].fillna( median_days[symptom], inplace=True)
    attributes[f'enroll_days_with_{symptom}'] = np.where(attributes[f'enroll_days_with_{symptom}'] > 100, median_days[symptom], \
                                                         attributes[f'enroll_days_with_{symptom}'])
    
median_age = attributes.loc[attributes['enroll_patient_age'] < 100, 'enroll_patient_age'].median()
attributes["enroll_patient_age"] = np.where(attributes["enroll_patient_age"] > 100, median_age, attributes['enroll_patient_age'])

continuous_var = ['enroll_patient_age', 'enroll_patient_temperature', 'enroll_days_with_cough', 'enroll_days_with_shortness_of_breath', 'enroll_days_with_fever']
categorical_var = ['enroll_travel_history', 'enroll_contact_with_confirmed_covid_case',
           'enroll_health_worker', 'enroll_fever', 'enroll_cough', 'enroll_shortness_of_breath']

# Normalizing it
for var in continuous_var:
    attributes[var] = (attributes[var] - attributes[var].mean()) / attributes[var].std()
    
for var in categorical_var:    
    le = LabelEncoder()
    attributes[var] = le.fit_transform(attributes[var])

In [31]:
attributes[continuous_var + categorical_var].head()

,enroll_patient_age,enroll_patient_temperature,enroll_days_with_cough,enroll_days_with_shortness_of_breath,enroll_days_with_fever,enroll_travel_history,enroll_contact_with_confirmed_covid_case,enroll_health_worker,enroll_fever,enroll_cough,enroll_shortness_of_breath
0,-0.590524,0.936583,-0.391203,-0.192757,-0.346219,0,0,1,0,0,0
1,-0.072324,0.695824,0.882046,-0.192757,-0.346219,0,0,0,0,1,0
2,0.223791,0.455065,-0.391203,-0.192757,-0.346219,0,0,0,0,0,0
3,1.556305,0.856330,-0.391203,-0.192757,-0.346219,0,0,0,0,0,0
4,0.223791,0.695824,-0.391203,-0.192757,-0.346219,0,0,0,0,0,0


### Post Pre-Processing, Save the `attributes.csv` as a different file

In [32]:
attributes.to_csv('/data/wiai-facility/processed/attributes_context_processed.csv', index=False)